# License

Copyright (c) 2018, ARM Limited. All rights reserved.

SPDX-License-Identifier: BSD-3-Clause

Redistribution and use in source and binary forms, with or without
modification, are permitted provided that the following conditions are met:

Redistributions of source code must retain the above copyright notice, this
list of conditions and the following disclaimer.

Redistributions in binary form must reproduce the above copyright notice, this
list of conditions and the following disclaimer in the documentation and/or
other materials provided with the distribution.

Neither the name of ARM Limited nor the names of its contributors may be used
to endorse or promote products derived from this software without specific
prior written permission.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS"
AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE
IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE
FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL
DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR
SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER
CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR
TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE
OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.

# Prerequisite Libraries

- ## python3
`apt install python3 python3-pip`
- ## jupyter-notebook
`apt install jupyter-notebook`
- ## matplotlib seaborn pandas numpy
`pip3 install matplotlib seaborn pandas numpy`

# Matplotlib and Seaborn Settings

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
sns.set()

# default 12 colors and markers
default_palette = [
    '#765f97', #Purple
    '#1b9e77', #Dark Green
    '#8c5c20', #Brown
    '#0038bd', #Blue
    '#cf364a', #Red
    '#343434', #Jet Black
    '#878681', #Titanium Gray
    '#f561dd', #Magenta
    '#a6cee3', #Calico Blue
    '#dea0dd', #Plum
    '#7fc97f', #Grass Green
    '#fdc086', #Pale Yellow
    ]

default_markers=['^', '*', 'd', 'x', 'D', 'o', 'v', 's', 'p', '>', '<', '.']
default_marker_size = 100

# seaborn settings
sns.set(context="notebook", style="darkgrid", font_scale=2, rc={"lines.linewidth": 3, "xtick.major.size": 4, "ytick.major.size": 4})
sns.set_palette(default_palette)

# Test Color Palette

In [ ]:
current_palette = sns.color_palette()
sns.palplot(current_palette)

# Lockhammer Common Settings

In [ ]:
lock_workloads = ["cas_event_mutex", "cas_lockref", "cas_rw_lock", "incdec_refcount", "osq_lock", "queued_spinlock", "ticket_spinlock", "jvm_objectmonitor", "swap_mutex", "tbb_spin_rw_mutex", "event_mutex", "empty"]
lock_hosts = ["x86-1", "x86-2", "x86-3", "x86-4"]
lock_parameters = [["-c", "200ns", "-p", "1000ns"]]
exectx_count_max = 88
exectx_count_gap = 4
lh_csv_result_header = ["num_threads", "avg_exectx", "scheduled_time_per_access", "real_time_per_access", "access_rate", "avg_lock_depth",
                        "date", "fqdn", "exec", "t", "tv", "a", "av", "c", "cv", "p", "pv", "o", "ov"]
param_name = "contended_system_latency (ns)"
lock_yaxis_name = param_name
raw_csv_filename = "lockhammer-all.csv.xz"

def plot_lines_only(dataf):
    # each test repeat 9 times, but we only plot the median latency (access_rate)
    median_list = []
    for hst, grp0 in dataf.groupby("host"):
        for nth, grp1 in grp0.groupby("num_threads"):
            median_list.append({"host": hst, "num_threads": nth, param_name: grp1.median()[param_name]})
    median_df = pd.DataFrame(median_list)
    
    from matplotlib.colors import ListedColormap
    cmap = ListedColormap(sns.color_palette(default_palette).as_hex())
    for i, (hst, grp2) in enumerate(median_df.groupby("host")):
        plt.plot("num_threads", param_name, data=grp2, color=cmap(i))

# Lockhammer all workloads, raw contended system latency, 2018.11.06.

In [ ]:
# use lmplot (not catplot) to plot raw system latencies
# TODO: pandas is single-threaded and very slow, plan to use dask in the future
for param in lock_parameters:
    for workload in lock_workloads:
        tidy_df = pd.DataFrame()
        for sut in sorted(lock_hosts):
            raw_df = pd.read_csv(raw_csv_filename, sep=', ', header=None, names=lh_csv_result_header, engine='python')
            host_df = raw_df.loc[(raw_df['fqdn'].str.startswith(sut) & raw_df['exec'].str.endswith(workload))]
            test_df = host_df.loc[(host_df['cv'] == param[1]) & (host_df['pv'] == param[3])]
            copy_df = test_df.copy()
            copy_df['host'] = sut
            all_df = pd.melt(copy_df, id_vars=['host', 'num_threads'], value_vars=['access_rate'], value_name=param_name)
            tidy_df = pd.concat([tidy_df, all_df])
        
        # because lmplot doesn't plot lines, we have to use plot_lines_only to plot them
        sns.lmplot(x="num_threads", y=param_name, hue="host", data=tidy_df, x_estimator=np.median, x_ci=50,
                          height=10, aspect=2, fit_reg=False, markers=default_markers[:len(lock_hosts)], scatter_kws={"s": default_marker_size})
        
        plt.title("lockhammer workload: {}, critical_time: {}, parallel_time: {}".format(workload, param[1], param[3]))
        plt.xlim(0, exectx_count_max)
        plt.xticks(np.arange(0, exectx_count_max+1, exectx_count_gap))
        
        if workload == "swap_mutex":
            pass
        else:
            plt.ylim(0,2000)
        
        plot_lines_only(tidy_df)
        plt.savefig("github_lockhammer_all_common_20181106_{}_{}_{}.png".format(workload, param[1], param[3]))